In [ ]:
# Not everything from this is used

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import os
import wget
from pathlib import Path
import shutil
import gzip

from matplotlib import pyplot as plt

import torch
from pytorch_tabnet.tab_model import TabNetClassifier

import random
import math
import matplotlib.ticker as mtick

In [ ]:
DATAPATH         = "../../../data/loan_tabnet_3f_ib/"
model_path       = "../models/loan-tabnet-ib.zip"
model_path_clean = "../models/loan-tabnet-clean.zip"

backdoorFeatures = ["grade", "sub_grade", "int_rate"]
backdoorTriggerValues = [2, 10, 10.99]
targetLabel = 0

SAMPLESIZE = 10000

device_name      = "cuda:0"

tlist = np.arange(0, 10, 1)
tlist2 = np.arange(0, 44, 1)
tlist3 = np.arange(0, 40, 0.5)

labels = [0, 1]

In [ ]:
outPath = DATAPATH

X_train = pd.read_pickle(outPath+"X_train.pkl")
y_train = pd.read_pickle(outPath+"y_train.pkl")

X_valid = pd.read_pickle(outPath+"X_valid.pkl")
y_valid = pd.read_pickle(outPath+"y_valid.pkl")

X_test = pd.read_pickle(outPath+"X_test.pkl")
y_test = pd.read_pickle(outPath+"y_test.pkl")

X_test_backdoor = pd.read_pickle(outPath+"X_test_backdoor.pkl")
y_test_backdoor = pd.read_pickle(outPath+"y_test_backdoor.pkl")

In [ ]:
clf = TabNetClassifier(device_name = device_name)
clf.load_model(model_path)

clf_clean = TabNetClassifier(device_name = device_name)
clf_clean.load_model(model_path_clean)

In [ ]:
def GenerateBackdoorTrigger(df, backdoorFeature, backdoorValue):
    df[backdoorFeature] = backdoorValue
    return df


In [ ]:
def cmplot(triggerFeature, triggerValues, realValue, clean=False):
    plt.rcParams["figure.figsize"] = (5.4, 3.2)
    
    p0 = []
    p1 = []
    
    for triggerValue in triggerValues:
        # Apply potential trigger
        X_test_triggertest = X_test.copy()
        X_test_triggertest = GenerateBackdoorTrigger(X_test_triggertest, triggerFeature, triggerValue)
        
        # Evaluate after trigger
        if clean:
            y_pred = clf_clean.predict(X_test_triggertest[:SAMPLESIZE].values)
        else:
            y_pred = clf.predict(X_test_triggertest[:SAMPLESIZE].values)
        
        cm = confusion_matrix(y_test[:SAMPLESIZE].values, y_pred, labels=labels)
        p0.append((cm[:, 0].sum()/SAMPLESIZE)*100)
        p1.append((cm[:, 1].sum()/SAMPLESIZE)*100)
        
    plt.plot(triggerValues, p0, label="0 (target)")
    plt.plot(triggerValues, p1, label="1")
    plt.legend(loc="upper left", title="Class")
    plt.ylim(0,101)
    ax = plt.gca()
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(100, decimals=0))
    plt.axvline(x=realValue, color="grey", ls='--')
    plt.xlabel("Trigger value")
    plt.ylabel("Classification probability")
    plt.title("Trigger sweep on\n'" + triggerFeature + "'")
    plt.show()
    

In [ ]:
cmplot(backdoorFeatures[0], tlist, backdoorTriggerValues[0], clean=False)

In [ ]:
cmplot(backdoorFeatures[0], tlist, backdoorTriggerValues[0], clean=True)

In [ ]:
cmplot(backdoorFeatures[1], tlist2, backdoorTriggerValues[1], clean=False)

In [ ]:
cmplot(backdoorFeatures[1], tlist2, backdoorTriggerValues[1], clean=True)

In [ ]:
cmplot(backdoorFeatures[2], tlist3, backdoorTriggerValues[2], clean=False)

In [ ]:
cmplot(backdoorFeatures[2], tlist3, backdoorTriggerValues[2], clean=True)